In [1]:
import os
import glob
from tqdm import tqdm
import json
from exif import Image, DATETIME_STR_FORMAT
from datetime import datetime
import math

In [2]:
os.listdir()

['.git',
 '.gitignore',
 'fix.py',
 'output',
 'README.md',
 'takeout_folders',
 'test.ipynb']

In [3]:
# https://auth0.com/blog/read-edit-exif-metadata-in-photos-with-python/

In [4]:
if not os.path.exists("output"):
    os.mkdir("output")
if not os.path.exists("takeout_folders"):
    os.mkdir("takeout_folders")

In [5]:
failures = []

In [6]:
# https://stackoverflow.com/a/52371976/3675086
def deg_to_dms(deg, type='lat'):
    decimals, number = math.modf(deg)
    d = int(number)
    m = int(decimals * 60)
    s = (deg - d - m / 60) * 3600.00
    compass = {
        'lat': ('N','S'),
        'lon': ('E','W')
    }
    compass_str = compass[type][0 if d >= 0 else 1]
    return {
        'dms': (abs(d), abs(m), abs(s)),
        'ref': compass_str
    }

In [7]:
def attach_metadata(files):
    for file in files:
        try:
            with open(f'{file}.json') as json_file:
                metadata = json.load(json_file)
        except FileNotFoundError:
            failures.append(file)
        else:
            with open(file, 'rb') as img_file:
                img = Image(img_file)

            timestamp = datetime.utcfromtimestamp(int(metadata['photoTakenTime']['timestamp'])).strftime(DATETIME_STR_FORMAT) # TODO SOMETHING WRONG HERE 

            img.datetime = timestamp

            # Try to set original date, and if it fails cuz no original date was ever set on the photo, then just skip 
            try:
                img.datetime_original = timestamp
            except Exception as e:
                pass

            if 'mobileUpload' in metadata['googlePhotosOrigin'] and 'deviceType' in metadata['googlePhotosOrigin']['mobileUpload']:
                if metadata['googlePhotosOrigin']['mobileUpload']['deviceType'] == "IOS_PHONE":
                    img.make = "Apple"
                    img.model = "iPhone 6"


            # Only add lat and lon to metadata if they are available from original photo 
            if metadata['geoData']['latitude'] != 0.0 and metadata['geoData']['longitude'] != 0.0:
                # convert decimal to degree, minute, second
                lat = deg_to_dms(metadata['geoData']['latitude'])
                lon = deg_to_dms(metadata['geoData']['longitude'])


                img.gps_latitude = lat['dms']
                img.gps_latitude_ref = lat['ref']
                img.gps_longitude = lon['dms']
                img.gps_longitude_ref = lon['ref']

            # print(os.getcwd())
            with open(f'../../../../../output/{file}', 'wb+') as new_image_file:
                new_image_file.write(img.get_file())

In [9]:
os.chdir("takeout_folders")
for folder in os.listdir():
    print(f"Fixing folder {folder}")

    backwards_traverse = "../"
    
    # Traverse down extra paths if necessary to get to year folders 
    if os.path.exists(f'{folder}/Takeout'):
        os.chdir(f"{folder}/Takeout/Google Photos")
        backwards_traverse = "../../../"

    photo_folders = [folders[1] for folders in os.walk("./")][0]
    
    for photo_folder in tqdm(photo_folders):
        os.chdir(photo_folder)

        # TODO not case insensitive
        # TODO add png

        attach_metadata(glob.glob("*.JPG"))

        # attach_metadata(glob.glob("*.MOV"))

        os.chdir("../")
    os.chdir(backwards_traverse)

Fixing folder takeout-20230525T041716Z-001
['Photos from 2013', 'Photos from 2014', 'Photos from 2015', 'Photos from 2016', 'Photos from 2017']


  0%|          | 0/5 [00:00<?, ?it/s]

Photos from 2013
Photos from 2014
Photos from 2015


 60%|██████    | 3/5 [00:00<00:00,  6.38it/s]

Photos from 2016


...

  0%|          | 0/2 [00:00<?, ?it/s]

Photos from 2016


100%|██████████| 2/2 [00:00<00:00,  5.00it/s]

Photos from 2017


100%|██████████| 2/2 [00:00<00:00,  4.92it/s]


In [10]:
len(failures)

357